### Loading data

In **_code cell 0.3_**, you can load all survey data you have collected during the field practicum. 
These include:
- FDEM_surveydata: full survey dataset of the Bottelare testfield;
- FDEM_transect: extract of the survey dataset along a reference transect. You can use this to test analytical procedures, which you can then deploy across the full survey dataset;
- sampling_data: calibration and validation sample dataset, collected as two independent stratified random sample sets (samples 1 - 10 =  calibration; 11 - 15 =  validation).

The sampling_data dataset includes the x and y coordinates of the samples along with their ID's, and analytical data on:

- bulk density in g/cm^3   (`bd [g/cm3]`);
- volumetric water content in % (`vwc [%]`);
- clay content in %   (`clay [%]`);
- soil organic matter in %   (`SOM [%]`);
- gravimetric water content (`gmc [%]`);
- silt content in %   (`silt [%]`);
- sand content in %   (`sand [%]`);
- porosity [dimensionless]   (`por [-]`);
- total organic carbon in g/kg   (`SOC [g/kg]`);

- hydraprobe volumetric water content in %   (`hydra_vwc [%]`)
- hydraprobe raw EC in mS/m   (`hydra_ec [mS/m]`)
- hydraprobe soil pore water EC in mS/m   (`hydra_ecp [mS/m]`)
- hydraprobe soil temperature in C   (`hydra_tmp [C]`)
- hydraprobe temperature corrected EC in mS/m   (`hydra_ect [mS/m]`)
- hydraprobe dielectric permittivity [dimensionless]   (`hydra_er [-]`).

The column names are shown between the brackets above. 
You can use these to access the desired data values in the `ds` dataframe (e.g., for bulk density, the values are in the `ds['bd [g/cm3]]` column).